## Generate images for evaluation

In [ ]:
import json, os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
from tqdm import *
from PIL import Image
from diffusers.pipelines import BlipDiffusionPipeline
from diffusers.utils import load_image
from utils.dataset_info import get_subjects_prompts_info

# Single Subject Generation
single_subject = ""                            # "backpack"
# Single Prompt Generation
single_prompt = []                            # e.g. ["a {0} {1} near the pool"]

num_generation = 4

output_path = "../../outputs/subjects/blip_diffusion"
dataset_path = "../../pcs_dataset"
dataset_info_path = f"{dataset_path}/info.json"
os.makedirs(output_path, exist_ok=True)

negative_prompt = "over-exposure, under-exposure, saturated, duplicate, out of frame, lowres, cropped, worst quality, low quality, jpeg artifacts, morbid, mutilated, out of frame, ugly, bad anatomy, bad proportions, deformed, blurry, duplicate"
blip_diffusion_pipe = BlipDiffusionPipeline.from_pretrained(
    "Salesforce/blipdiffusion", torch_dtype=torch.float16
).to("cuda")

prompts_info = get_subjects_prompts_info(dataset_info_path)
if single_subject != "":
    prompts_info = {f"{single_subject}": prompts_info[single_subject]}


for subject in prompts_info:

    if subject not in os.listdir(output_path):

        if len(single_prompt):
            prompts = single_prompt
        else:
            prompts = prompts_info[subject]["prompts"]

        for prompt in prompts:
            text_prompt_input = prompt.replace("a {0} {1}", "")

            for i in range(0, num_generation):
                cond_image = Image.open(os.path.join(dataset_path, "subjects", subject, f"0{i}.jpg"))

                image = blip_diffusion_pipe(
                    text_prompt_input,
                    cond_image,
                    prompts_info[subject]["class"],
                    prompts_info[subject]["class"],
                    guidance_scale=7.5,
                    num_inference_steps=50,
                    neg_prompt=negative_prompt,
                    height=512,
                    width=512,
                ).images

                save_path = os.path.join(output_path, subject, f"{text_prompt_input}")
                os.makedirs(save_path, exist_ok=True)

                image[0].save(os.path.join(save_path, f"{i:04d}.jpg"))
                
        print(f"Finished blip diffusion in subject: {subject}!")


## Evaluation

Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [17]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import numpy as np
from utils.clip_eval import CLIPEvaluator, evaluate_i2i, evaluate_t2i
import re
from utils.dataset_info import get_subjects_prompts_info

In [18]:
def extract_id(folder_name):
    match = re.search(r'\d+', folder_name)
    if match:
        return int(match.group())
    return None

def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data

In [ ]:
outputs_path = "../../outputs/subjects/blip_diffusion"
eval_res_path = "../../eval_results/subjects/blip_diffusion"
dataset_path = "../../pcs_dataset/subjects"
dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)

subjects_list = os.listdir(outputs_path)

evaluation_res = dict()
for subject in subjects_list:
    print("process subject: ", subject)

    res_for_each_subject =dict()

    for sub_dir in os.listdir(os.path.join(outputs_path, subject)):
        print("prompt: ", sub_dir)

        prompt = prompts[prompts_info[subject]["prompt_type"]][extract_id(sub_dir)]
        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, subject, sub_dir)):
            generate_img_path = os.path.join(outputs_path, subject, sub_dir, generate_img_name)
            res_for_each_prompt[generate_img_name] = [evaluate_i2i(generate_img_path, os.path.join(dataset_path, subject)), evaluate_t2i(generate_img_path, prompt)]
        
        res_for_each_subject[prompt] = res_for_each_prompt

        print(res_for_each_prompt)

    evaluation_res[subject] = res_for_each_subject

os.makedirs(eval_res_path, exist_ok=True)

with open(os.path.join(eval_res_path, "evaluation_results.json"), "w") as json_file:
    json.dump(convert_to_native(evaluation_res), json_file, indent=4)
